In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip3 install sktime neuralforecast pmdarima darts numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
os.listdir()

['.config', 'engagement-prediction', 'drive', 'sample_data']

In [8]:
import os
os.chdir('engagement-prediction/src')

FileNotFoundError: [Errno 2] No such file or directory: 'engagement-prediction/src'

In [5]:
storage_path='/content/drive/MyDrive/thesis/'

In [6]:
import pandas as pd
import numpy as np
import os
from metrics_forecast_class import TimeSeriesEvaluator,MultiSeriesEvaluator
h=6
window=24
sources=['instagram-content','twitter-tweets','tiktok-content','facebook-content','youtube-content',
 'tiktok-content-oauth','instagram-content-oauth','tiktok-content-public']
file_path=os.path.join(storage_path,'data','influencer_sample010525.csv')
df= pd.read_csv(file_path)
#df= pd.read_csv('~/rug/thesis/data/influencer_sample010525.csv')
df=df.query('source in @sources').reset_index(drop=True)
df['published_date'] = pd.to_datetime(df['publishedDate'])
df
df_temp= df.query('channel_uid=="51b52b775df43f1ea0abf5db8789446d"')[['published_date', 'video_plays']].dropna()
df_temp = df_temp.set_index('published_date').sort_index()
y= df_temp['video_plays']
y = y.sort_index()
y_pos = y.reset_index(drop=True)

#y = y.asfreq('D')
y

ModuleNotFoundError: No module named 'metrics_forecast_class'

In [ ]:
df['source'].value_counts()

source
instagram-content          136584
twitter-tweets              50484
tiktok-content              35057
facebook-content            10885
youtube-content              2949
tiktok-content-oauth         1602
instagram-content-oauth       734
tiktok-content-public          62
Name: count, dtype: int64

In [ ]:
df.groupby('channel_uid').count()['publishedDate'].sort_values(ascending=False).head(10)

channel_uid
26bf145cbe9c3a10a53c619f05aa750f    32674
1058025cd111366f9d4d4af89ce1d8d0    24051
954328679b923a4fa7af327a2cb5b84d    13640
da9acec7669f3db79050175948471cde     9714
e04c6702e00a321a8f98624ffe7fd23e     6232
6b9e43bfbb843f07b99ec55767b87e67     5466
24a13f11774b37b2bb94e9ab25327bf7     4884
e6981573da773e0eb0493a0cf142e27c     4274
428271c836453fc2a6fd57748371c14d     3608
1fc2d888a1d63daea81ab67327629a64     3478
Name: publishedDate, dtype: int64

In [ ]:
ids_test=df.query("""source in ['instagram-content','youtube-content','tiktok-content','facebook-content']""")\
    ['channel_uid'].drop_duplicates().sample(20, random_state=1).values
df_temp=df.query('channel_uid in @ids_test')
df_temp['source'].unique()

array(['tiktok-content', 'tiktok-content-oauth', 'instagram-content',
       'facebook-content', 'youtube-content', 'tiktok-content-public'],
      dtype=object)

In [ ]:
multi_eval = MultiSeriesEvaluator(df_temp, date_col='published_date', channel_col='channel_uid')

In [ ]:

multi_eval.run(h=6, initial_window=24, step_length=6,models=['MA(4)','AR(4)'],metric_cols=['potentialReach','engagements','video_views','comments','shares'])

summary_df = multi_eval.get_summary()

/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Evaluating models:   0%|          | 0/120 [00:00<?, ?it/s]2025-05-08 12:04:21,469 Evaluating channel: e4786c9ac7d23a40aa6840e8d817642b
2025-05-08 12:04:21,482 Evaluating metric: potentialReach
2025-05-08 12:04:21,483 Evaluating AutoREG(4) for e4786c9ac7d23a40aa6840e8d817642b/potentialReach
Evaluating models:   1%|          | 1/120 [00:00<01:00,  1.95it/s]2025-05-08 12:04:21,982 1 of 120
2025-05-08 12:04:21,983 Evaluating AR(4) for e4786c9ac7d23a40aa6840e8d817642b/potentialReach
/home/esteban/rug/thesis/thesis/venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/esteban/rug/thesis/

In [ ]:
multi_eval.get_summary().query('channel_uid=="e4786c9ac7d23a40aa6840e8d817642b"')

,model_name,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,test_MeanSquaredError,channel_uid,metric_name,source
0,AutoREG(4),2.485952e-11,4.227810e-16,2.640873e-11,e4786c9ac7d23a40aa6840e8d817642b,potentialReach,tiktok-content
1,AR(4),1.129709e-03,1.921273e-08,1.300282e-03,e4786c9ac7d23a40aa6840e8d817642b,potentialReach,tiktok-content
2,MA(4),4.824575e-06,8.205060e-11,4.824575e-06,e4786c9ac7d23a40aa6840e8d817642b,potentialReach,tiktok-content
3,AutoREG(4),1.421318e+02,8.506997e-01,1.709440e+02,e4786c9ac7d23a40aa6840e8d817642b,engagements,tiktok-content
4,AR(4),1.431922e+02,8.393631e-01,1.706983e+02,e4786c9ac7d23a40aa6840e8d817642b,engagements,tiktok-content
5,MA(4),1.492030e+02,8.638204e-01,1.779779e+02,e4786c9ac7d23a40aa6840e8d817642b,engagements,tiktok-content


In [2]:
import datetime as dt
output_path=os.path.join(storage_path,'cache',f"summary_df_{dt.datetime.now():%y%m%d%H%M}.csv")
summary_df.to_csv(output_path, index=False)
summary_df

NameError: name 'os' is not defined